In [1]:
%cd src

/home/ubuntu/rebel/src


/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from hydra import compose, initialize
import pytorch_lightning as pl
from pl_data_modules import BasePLDataModule
from pl_modules import BasePLModule
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer
from pytorch_lightning.core.saving import _load_from_checkpoint
from typing import cast
import torch
from pytorch_lightning.utilities.migration.utils import _pl_migrate_checkpoint
from pathlib import Path
from pytorch_lightning.core.saving import _load_state

checkpoint_path = "/home/ubuntu/rebel/outputs/2024-08-21/17-25-14/experiments/default_name/last.ckpt"
seed = 42
config_name = 'google/mt5-base'
tokenizer_name = 'google/mt5-base'

/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
initialize(config_path="./conf")
conf = compose(config_name="root", overrides=[f"checkpoint_path={checkpoint_path}"])

/tmp/ipykernel_692398/2836277268.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="./conf")


In [4]:
config = AutoConfig.from_pretrained(
    conf.config_name if conf.config_name else conf.model_name_or_path,
    decoder_start_token_id = 0,
    early_stopping = False,
    no_repeat_ngram_size = 0,
)

special_tokens = [
    "<triplet>",
    "<obj>",
    "<subj>",
]
    
tokenizer_kwargs = {
    "use_fast": True,  # Always use fast tokenizer for better compatibility
    "additional_special_tokens": special_tokens, 
    "legacy": False,  # For mt5
}

tokenizer = AutoTokenizer.from_pretrained(
    conf.tokenizer_name if conf.tokenizer_name else conf.model_name_or_path,
    **tokenizer_kwargs
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    conf.model_name_or_path,
    config=config,
)
# if not conf.finetune:
model.resize_token_embeddings(len(tokenizer))
model.eval()

/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


MT5ForConditionalGeneration(
  (shared): Embedding(250103, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250103, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [5]:
# # data module declaration
# pl_data_module = BasePLDataModule(conf, tokenizer, model)

# # main module declaration
# pl_module = BasePLModule(conf, config, tokenizer, model)
# device = torch.device("cuda")

In [6]:
checkpoint = torch.load(
    conf.checkpoint_path,
    map_location=torch.device('cuda:0'),
)
# checkpoint = _pl_migrate_checkpoint(
#     checkpoint, checkpoint_path=(conf.checkpoint_path if isinstance(conf.checkpoint_path, (str, Path)) else None)
# )
# model_state_dict = 

# Remove the 'model.' prefix from the keys if present
model_state_dict = {k.replace('model.', ''): v for k, v in checkpoint['state_dict'].items()}
model.load_state_dict(model_state_dict)
model = model.to(torch.device('cuda:0'))

/tmp/ipykernel_692398/2773581032.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(


In [7]:

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = (
        text
        .replace("<obj>", " <obj> ")
        .replace("<subj>", " <subj> ")
        .replace("<triplet>", " <triplet> ")
        .replace("<s>", "")
        .replace("</s>", "")
        .replace("<pad>", "")
        .strip()
    )
    for i in range(100):
        text = text.replace(f"<extra_id_{i}>", "")
    current = 'x'
    for token in text.split():
        token = token.strip()
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if not subject.strip() and not relation.strip() and not object_.strip():
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    triplets = [item for item in triplets if item["head"] and item["type"] and item["tail"]]
    return triplets

In [8]:
# def on_train_batch_end(
#     self,
#     trainer,
#     pl_module,
#     outputs,
#     batch,
#     batch_idx,
# ) -> None:
#     wandb_table = wandb.Table(columns=["Source", "Pred", "Gold"])
#     # pl_module.logger.info("Executing translation callback")
#     if (batch_idx + 1) % self.logging_batch_interval != 0:  # type: ignore[attr-defined]
#         return
#     labels = batch.pop("labels")
#     gen_kwargs = {
#         "max_length": pl_module.hparams.val_max_target_length
#         if pl_module.hparams.val_max_target_length is not None
#         else pl_module.config.max_length,
#         "early_stopping": False,
#         "no_repeat_ngram_size": 0,
#         "num_beams": pl_module.hparams.eval_beams if pl_module.hparams.eval_beams is not None else pl_module.config.num_beams,
#     }
#     pl_module.eval()

#     decoder_inputs = torch.roll(labels, 1, 1)[:,0:2]
#     decoder_inputs[:, 0] = 0
#     generated_tokens = pl_module.model.generate(
#         batch["input_ids"].to(pl_module.model.device),
#         attention_mask=batch["attention_mask"].to(pl_module.model.device),
#         decoder_input_ids=decoder_inputs.to(pl_module.model.device),
#         **gen_kwargs,
#     )
#     # in case the batch is shorter than max length, the output should be padded
#     if generated_tokens.shape[-1] < gen_kwargs["max_length"]:
#         generated_tokens = pl_module._pad_tensors_to_max_len(generated_tokens, gen_kwargs["max_length"])
#     pl_module.train()
#     decoded_preds = pl_module.tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

#     if pl_module.hparams.ignore_pad_token_for_loss:
#         # Replace -100 in the labels as we can't decode them.
#         labels = torch.where(labels != -100, labels, pl_module.tokenizer.pad_token_id)

#     decoded_labels = pl_module.tokenizer.batch_decode(labels, skip_special_tokens=False)
#     decoded_inputs = pl_module.tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=False)

In [27]:
def _predict(model, tokenizer, text):
    gen_kwargs = {
        "max_length": 128,
        "early_stopping": False,
        "length_penalty": 0,
        "no_repeat_ngram_size": 0,
        "num_beams": 3,
    }
    
    model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')
    batch_size = model_inputs['input_ids'].shape[0]
    decoder_inputs = torch.tensor([[0, 250100] for _ in range(batch_size)])
    generated_tokens = model.generate(
        model_inputs["input_ids"].to(model.device),
        attention_mask=model_inputs["attention_mask"].to(model.device),
        decoder_input_ids=decoder_inputs.to(model.device),
        **gen_kwargs,
    )
    
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
    return decoded_preds

predict = lambda text: _predict(model, tokenizer, text)

In [29]:
text = 'پایین سرست، روستایی است از توابع بخش گتاب شهرستان بابل در استان مازندران ایران.</s>'
# print(predict(text))
decoded_preds = predict(text.strip())
extract_triplets(decoded_preds[0])

[{'head': 'پایین سرست', 'type': 'کشور', 'tail': 'ایران'},
 {'head': 'بخش گتاب',
  'type': 'موقعیت در تقسیمات کشوری',
  'tail': 'شهرستان بابل'},
 {'head': 'بخش گتاب', 'type': 'کشور', 'tail': 'ایران'},
 {'head': 'شهرستان بابل', 'type': 'زیربخش', 'tail': 'بخش گتاب'},
 {'head': 'شهرستان بابل',
  'type': 'موقعیت در تقسیمات کشوری',
  'tail': 'استان مازندران'},
 {'head': 'شهرستان بابل', 'type': 'کشور', 'tail': 'ایران'},
 {'head': 'استان مازندران', 'type': 'زیربخش', 'tail': 'شهرستان بابل'},
 {'head': 'استان مازندران',
  'type': 'موقعیت در تقسیمات کشوری',
  'tail': 'ایران'}]